In [0]:
import torch
import torch.nn as nn

In [0]:
class Ffunction(nn.Module):
  def __init__(self, input_size):
    super(Ffunction, self).__init__()
    self.input_size = input_size
    self.lstm = nn.LSTM(input_size, 64)
    self.fullycon1 = nn.Linear(64, 64)
    nn.init.orthogonal(self.fullycon1.weight)
    self.fullycon2 = nn.Linear(64, 64)
    nn.init.orthogonal(self.fullycon2.weight)
    self.policy = nn.Linear(64, 5)
    nn.init.orthogonal(self.policy.weight)
    self.value = nn.Linear(64, 1)
    nn.init.orthogonal(self.value.weight)

    self.beta = 0.01

  def forward(self, x): #x = input
    result, _ = self.lstm(x.view(-1, 1, self.input_size))
    result = nn.functional.elu(self.fullycon1(result.view(-1, 64)))
    result = nn.functional.elu(self.fullycon2(result))
    return nn.functional.softmax(self.policy(result)), self.value(result)

  def getA2Closs(self, actions, states, rewards):
    probs, values = self.forward(states)
    advantage = rewards - values
    actions_one_hot = torch.zeros(batch_size, 5)
    actions_one_hot[torch.arange(batch_size), actions.view(batch_size)] = 1
    logs = torch.log(torch.sum(probs.view(batch_size, 5) * actions_one_hot.view(batch_size, 5), (1)))
    logloss = logs * advantage.view(batch_size)
    entropyloss = -torch.sum(probs * torch.log(probs), (1))

    return logloss + self.beta * entropyloss

In [4]:
agent = Ffunction(326)
opt = torch.optim.Adam(agent.parameters(), betas=(0.9, 0.999), eps=1 * 10 ** -8)
batch_size = 20

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  del sys.path[0]


In [0]:
for episodes in range(100):
  x_other = torch.FloatTensor([1] * 3) / 3
  x_other.requires_grad_(True)

  opt2 = torch.optim.SGD([x_other], lr=0.1)

  env = #env

  for step in range(max_steps):



    